

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/SENTENCE_GRAMMAR.ipynb)



# **Evaluate Sentence Grammar**

## 1. Colab Setup

In [ ]:
# Install java
!apt-get update -qq
!apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
!java -version

# This is only to setup PySpark and Spark NLP on Colab
# -p is for pyspark
# -s is for spark-nlp
# !bash colab_setup.sh -p 3.1.1 -s 3.0.0  
# by default they are set to the latest

!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/colab_setup.sh
!bash colab_setup.sh

# Update environmental variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]

In [3]:
import pandas as pd
import numpy as np
import os
import json
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

## 2. Start Spark Session

In [4]:
spark = sparknlp.start()

## 3. Select the model to use

In [8]:
#MODEL_NAME = 't5_small'
MODEL_NAME = 't5_base'

## 4 Examples to try on the model

In [10]:
text_list = ['Anna and Mike is going skiing and they is liked is', 'Anna and Mike like to dance']

## 5. Define the Spark NLP pipeline

The `T5 Transformer` model is able to perform 18 different tasks (ref.: [this paper](https://arxiv.org/abs/1910.10683)). To check the grammar in a sentence, we use the prefix `cola sentence:` in the model.

In [9]:
# Prefix to be used on the T5Transformer().setTask(<<prefix>>)
task_prefix = 'cola sentence:'

In [11]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("documents")

t5 = T5Transformer() \
    .pretrained(MODEL_NAME) \
    .setTask(task_prefix)\
    .setMaxOutputLength(200)\
    .setInputCols(["documents"]) \
    .setOutputCol("T5")

pipeline = Pipeline(stages=[document_assembler, t5])

t5_base download started this may take some time.
Approximate size to download 446 MB
[OK!]


## 6. Run the pipeline

In [12]:
# Fit on empty data frame (model is pretrained)
empty_df = spark.createDataFrame([['']]).toDF('text')
pipeline_model = pipeline.fit(empty_df)

# Create Light Pipeline
lmodel = LightPipeline(pipeline_model)

# Use the model to make predictions
res = lmodel.fullAnnotate(text_list)

## 7. Visualize the results

Using Light Pipeline:

In [18]:
for r in res:
    print(f"{r['documents'][0].result} => Grammar: {r['T5'][0].result}")

Anna and Mike is going skiing and they is liked is => Grammar: unacceptable
Anna and Mike like to dance => Grammar: acceptable


Using pipeline model:

In [22]:
# Send example texts to spark data frame
text_df = spark.createDataFrame(pd.DataFrame({'text': text_list}))

# Predict with the model
result = pipeline_model.transform(text_df)

In [21]:
result.select('text', 'T5.result').show(truncate=False)

+--------------------------------------------------+--------------+
|text                                              |result        |
+--------------------------------------------------+--------------+
|Anna and Mike is going skiing and they is liked is|[unacceptable]|
|Anna and Mike like to dance                       |[acceptable]  |
+--------------------------------------------------+--------------+

